### Wait, slow down there :)

Have you already trained your custom model?
To work through this notebook you will need a trained custom model.

If you still need to create your model, you can use the Document Intelligence Studio: https://documentintelligence.ai.azure.com/studio  
Just follow this guide: https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/how-to-guides/build-a-custom-model?view=doc-intel-4.0.0

If you need some sample forms, there are some very basic ones in \sample_forms

After creating the your trained custom model, come back here to write python code
to call your model's API endpoint to OCR a form and then parse the result JSON.


### Choose your library

In the first part of this notenbook we will use the azure.ai.formrecognizer library  
to make the API cal

In the second part of this notebook  
we will make the same API call using the newer azure-ai-documentintelligence library


### Install required libraries

Before we can make the API call to the Azure Document Intelligence Service  
we need the right libraries installed.

If you haven't already run the requirements.txt file you should do that first to install the needed libraries.


In [1]:
%pip install -r requirements.txt
#
# notice you will get the latest as of 3/14/24
# azure.ai.formrecognizer             version 3.3.2   which supports APIs    2.0, 2.1, 2022-08-31, 2023-07-31 (default)
# and azure-ai-documentintelligence   version 1.0.0b2 which supports API     2024-02-29-preview

Note: you may need to restart the kernel to use updated packages.


### Now read in secrets from the .env file

You will need these 4 pieces if information when we call our API:

1. The Model ID - Get it from the Document Intelligence Studio
2. The Document Intelligence Resource API key - Get it from the Document Intelligence Studio or the Azure Portal.
3. The Document Intelligence Resource Endpoint - Get it from the Document Intelligence Studio or the Azure Portal.
4. The URL for your test document that you want to OCR - get if from Azure Storage Explorer or The Azure Portal.  
   &nbsp;&nbsp;(If you need Azure Storage Explorer download it here : https://storageexplorer.com/ )

To avoid exposing your keys, endpoints or other secret information,  
you should store them external from your notebook and read them in programatically.

The code below expects them to be in an .env file like shown below:

DOCUMENT_INTELLIGENCE_ENDPOINT = "Your endpoint goes here"  
DOCUMENT_INTELLIGENCE_KEY = "Your key goes here"  
MODEL_ID = "Your Model ID goes here"  
DOCUMENT_URL = "The URL to your test document goes here"

I have provided a sample.env file.
Just fill in your values in the sample.env and then rename it to .env


In [2]:
import os
from dotenv import load_dotenv

#
# load my environment variables from .env file
load_dotenv()
#
# retrieve the values from the environment variables
#
# the endpoint to your model
endpoint = os.getenv("DOCUMENT_INTELLIGENCE_ENDPOINT")
#
# the API key to the model
key = os.getenv("DOCUMENT_INTELLIGENCE_KEY")
#
# your model ID
model_id = os.getenv("MODEL_ID")
#
# the URL to a document that you want to OCR
formUrl = os.getenv("DOCUMENT_URL")
#
if endpoint is None:
    raise ValueError("Please provide a valid endpoint in the .env file")
if key is None:
    raise ValueError("Please provide a valid key in the .env file")
if model_id is None:
    raise ValueError("Please provide a valid model ID in the .env file")
if formUrl is None:
    raise ValueError("Please provide a valid document URL in the .env file")

## Using the Form Recognizer Library

### Call the API

Now we will OCR the test form using the model by calling the endpoint
Note the variables endpoint, key, model_id, formUrl were populated in the previous cell


In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Make sure your document's type is included in the list of document types the custom model can analyze
poller = document_analysis_client.begin_analyze_document_from_url(model_id, formUrl)
result = poller.result()

### Parse the result

The Form has been OCR'd. The Result variable has lots of  
JSON that describes what the model identified.  
(It really has a lot of data)

Below is a set of generic parsing code that was given by the studio when I created the model.  
It is also shown in the SDK: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/formrecognizer/azure-ai-formrecognizer#analyze-documents-using-a-custom-model  
It does a generic unravling of the result JSON returned from the API call.


In [4]:
# this will extract the fields and their values from the document
for idx, document in enumerate(result.documents):
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has confidence {}".format(document.confidence))
    print("Document was analyzed by model with ID {}".format(result.model_id))
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print(
            "......found field of type '{}' with value '{}' and with confidence {}".format(
                field.value_type, field_value, field.confidence
            )
        )


# iterate over tables, lines, and selection marks on each page
# if you have checkboxes on your form this type of parse will be useful
for page in result.pages:
    print("\nLines found on page {}".format(page.page_number))
    for line in page.lines:
        print("...Line '{}'".format(line.content.encode("utf-8")))
    for word in page.words:
        print(
            "...Word '{}' has a confidence of {}".format(
                word.content.encode("utf-8"), word.confidence
            )
        )
    for selection_mark in page.selection_marks:
        print(
            "...Selection mark is '{}' and has a confidence of {}".format(
                selection_mark.state, selection_mark.confidence
            )
        )

# use this type of code if you modelled the data to extract as tables
for i, table in enumerate(result.tables):
    print("\nTable {} can be found on page:".format(i + 1))
    for region in table.bounding_regions:
        print("...{}".format(i + 1, region.page_number))
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index, cell.column_index, cell.content.encode("utf-8")
            )
        )
print("-----------------------------------")

--------Analyzing document #1--------
Document has type First_Form:First_Form
Document has confidence 0.994
Document was analyzed by model with ID First_Form
......found field of type 'string' with value '76 Sunset road' and with confidence 0.994
......found field of type 'string' with value 'Tyler Haywood' and with confidence 0.995
......found field of type 'string' with value 'Toledo' and with confidence 0.994
......found field of type 'string' with value 'OH' and with confidence 0.995

Lines found on page 1
...Line 'b'Please fill in the fields below''
...Line 'b'Name: Tyler Haywood''
...Line 'b'Address: 76 Sunset road''
...Line 'b'City: Toledo''
...Line 'b'State: OH''
...Word 'b'Please'' has a confidence of 0.995
...Word 'b'fill'' has a confidence of 0.993
...Word 'b'in'' has a confidence of 0.997
...Word 'b'the'' has a confidence of 0.994
...Word 'b'fields'' has a confidence of 0.995
...Word 'b'below'' has a confidence of 0.993
...Word 'b'Name:'' has a confidence of 0.995
...Word '

### Parse the models fields

But since we have created a custom model, we can chose to retrieve only those fields modelled when creating the model for and no more.  
My form had only 4 fields so I'll use part of the above supplied code as my example  
and pull the field names an values by looping through the document.fields.items().  
I I'll format my output as JSON.


In [6]:
for name, field in document.fields.items():
    field_value = field.value if field.value else field.content
    print('{"' + name + '" : "' + field_value + '"}')

{"Address" : "76 Sunset road"}
{"Name" : "Tyler Haywood"}
{"City" : "Toledo"}
{"State" : "OH"}


### Format the JSON to meet any expected format for downstream consumption!

************* End of Demo Code for azure.ai.formrecognizer *************


## Using the new Document Intelligence library

In this next section we will do the same thing as above, OCR a form.
But this time we will use the newer Document Intelligence library: azure-ai-documentintelligence.


### Install the required libaries

If we hadn't installed the requirement file already we would need to install azure-ai-documentintelligence.


In [ ]:
# Install required libraries

# %pip install azure-ai-documentintelligence


### Retrieve our Secrets

That code is not part of Document Intelligence and I already have my variables populated with secrets so we can move on.


### OCR a Form

We will use the azure-ai-documentintelligence libary.  
The code below sents out test form to the API endpoint.

Remember my endpoint , key, model_id and formUrl are already populated from values that
I read in from environment variables that I loaded in my .env file.


In [7]:
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, AnalyzeResult

#
# Initialize the Doc Intelligence client
document_intelligence_client = DocumentIntelligenceClient(
    endpoint, credential=AzureKeyCredential(key)
)
#
# set up call to the API
analyze_request = AnalyzeDocumentRequest(url_source=formUrl)
#
# make the call and wait for response
poller = document_intelligence_client.begin_analyze_document(
    model_id=model_id, analyze_request=analyze_request
)

# get the result of the call
result = poller.result()

### Parse the Results

Below is a generic parse routing for the results from the sdk: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/documentintelligence/azure-ai-documentintelligence#analyze-documents-using-a-custom-model


In [8]:
# I can extract information about the document
# and read the modelled fields
if result.documents:
    for idx, document in enumerate(result.documents):
        print(f"--------Analyzing document #{idx + 1}--------")
        print(f"Document has type {document.doc_type}")
        print(f"Document has document type confidence {document.confidence}")
        print(f"Document was analyzed with model with ID {result.model_id}")
        if document.fields:
            for name, field in document.fields.items():
                field_value = (
                    field.get("valueString")
                    if field.get("valueString")
                    else field.content
                )
                print(
                    f"......found field of type '{field.type}' with value '{field_value}' and with confidence {field.confidence}"
                )

# iterate over tables, lines, and selection marks on each page
for page in result.pages:
    print(f"\nLines found on page {page.page_number}")
    if page.lines:
        for line in page.lines:
            print(f"...Line '{line.content}'")
    if page.words:
        for word in page.words:
            print(f"...Word '{word.content}' has a confidence of {word.confidence}")
    if page.selection_marks:
        print(f"\nSelection marks found on page {page.page_number}")
        for selection_mark in page.selection_marks:
            print(
                f"...Selection mark is '{selection_mark.state}' and has a confidence of {selection_mark.confidence}"
            )

if result.tables:
    for i, table in enumerate(result.tables):
        print(f"\nTable {i + 1} can be found on page:")
        if table.bounding_regions:
            for region in table.bounding_regions:
                print(f"...{region.page_number}")
        for cell in table.cells:
            print(
                f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'"
            )
print("-----------------------------------")

--------Analyzing document #1--------
Document has type First_Form:First_Form
Document has document type confidence 0.994
Document was analyzed with model with ID First_Form
......found field of type 'DocumentFieldType.STRING' with value 'Toledo' and with confidence 0.994
......found field of type 'DocumentFieldType.STRING' with value 'Tyler Haywood' and with confidence 0.995
......found field of type 'DocumentFieldType.STRING' with value 'OH' and with confidence 0.995
......found field of type 'DocumentFieldType.STRING' with value '76 Sunset road' and with confidence 0.994

Lines found on page 1
...Line 'Please fill in the fields below'
...Line 'Name: Tyler Haywood'
...Line 'Address: 76 Sunset road'
...Line 'City: Toledo'
...Line 'State: OH'
...Word 'Please' has a confidence of 0.995
...Word 'fill' has a confidence of 0.993
...Word 'in' has a confidence of 0.997
...Word 'the' has a confidence of 0.994
...Word 'fields' has a confidence of 0.995
...Word 'below' has a confidence of 0.993

### Get Just our Modelled Fields

If that is all we want, the code is similar but not exactly the same as the azure.ai.formrecognizer  
I'll format my output to be JSON again.


In [19]:
if document.fields:
    for name, field in document.fields.items():
        field_value = (
            field.get("valueString") if field.get("valueString") else field.content
        )
        print('{"' + name + '"}:{"' + field_value + '"}')

{"City"}:{"Toledo"}
{"Name"}:{"Tyler Haywood"}
{"State"}:{"OH"}
{"Address"}:{"76 Sunset road"}


### Format as Needed by Downstream apps

If you have set of APIs sigatures or an expected ingestion format you can format the data as needed.

************* End of Demo Code for azure-ai-documentintelligence *************


### Links to Resources

#### For the SDKs

The current python SDK for azure-ai-documentintelligence (1.0.0b2 as of 3/14/24): https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/documentintelligence/azure-ai-documentintelligence  
The current python SDK for azure-ai-formrecognizer (3.3.X - Latest GA release as of 3/14/24) https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/formrecognizer/azure-ai-formrecognizer

#### More sample code for azure-ai-formrecognizer

For code snippets illustrating common patterns used in the Form Recognizer Python API see:
https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/formrecognizer/azure-ai-formrecognizer/samples#samples-for-client-library-versions-320-and-later
https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/formrecognizer/azure-ai-formrecognizer/samples

#### Documentation for azure-ai-formrecognizer

https://azuresdkdocs.blob.core.windows.net/$web/python/azure-ai-formrecognizer/latest/index.html

#### More sample code for azure-ai-documentintelligence

https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/documentintelligence/azure-ai-documentintelligence#examples  
https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/documentintelligence/azure-ai-documentintelligence/samples

#### Documentation for documentintelligence

https://azuresdkdocs.blob.core.windows.net/$web/python/azure-ai-documentintelligence/latest/index.html

#### What Models Features are available

https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept-model-overview?view=doc-intel-4.0.0

### How to do logging

https://learn.microsoft.com/en-us/azure/developer/python/sdk/azure-sdk-logging


### Whats Where?

You can find the supported API version for the latest programming language SDKs at the links below.
The links below are to the latest SDK versions and shows the relationship between supported Document Intelligence SDK and API versions:

#### Supported Language Azure SDK reference

• C#/.NET: 4.0.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://azuresdkdocs.blob.core.windows.net/$web/dotnet/Azure.AI.FormRecognizer/4.0.0/index.html  
• Java: 4.0.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://azuresdkdocs.blob.core.windows.net/$web/java/azure-ai-formrecognizer/4.0.0/index.html  
• JavaScript: 4.0.0&nbsp;&nbsp;&nbsp;&nbsp;https://azuresdkdocs.blob.core.windows.net/$web/javascript/azure-ai-form-recognizer/4.0.0/index.html  
• Python 3.2.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://azuresdkdocs.blob.core.windows.net/$web/python/azure-ai-formrecognizer/3.2.0/index.html

#### All of the above support these API versions

• 2023-10-31-preview&nbsp;https://learn.microsoft.com/en-us/rest/api/aiservices/document-models/build-model?view=rest-aiservices-2024-02-29-preview&branch=docintelligence&tabs=HTTP  
• v3.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://learn.microsoft.com/en-us/rest/api/aiservices/document-models/analyze-document?view=rest-aiservices-2023-07-31&preserve-view=true&tabs=HTTP  
• v2.1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://westus.dev.cognitive.microsoft.com/docs/services/form-recognizer-api-v2-1/operations/AnalyzeBusinessCardAsync  
• v2.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://westus.dev.cognitive.microsoft.com/docs/services/form-recognizer-api-v2/operations/AnalyzeLayoutAsync
